In [1]:
import os
import bids

In [2]:
fmriprep_dir = '/archive/data/SPINS/pipelines/bids_apps/fmriprep'
output = '/projects/jjeyachandra/fmriprep_qc_summary/output'

In [3]:
layout = bids.BIDSLayout(fmriprep_dir,derivatives=True,validate=False)

/mnt/tigrlab/projects/jjeyachandra/fmriprep_qc_summary/fmriprep_qc/lib/python3.6/site-packages/bids/layout/layout.py:571: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations (['/archive/data/SPINS/pipelines/bids_apps/fmriprep/derivatives']). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a dataset_description.json file in derivatives datasets).
  "datasets).".format(paths))


In [4]:
layout.root

'/archive/data/SPINS/pipelines/bids_apps/fmriprep'

In [5]:
#Get tasks and subjects
tasks = layout.get_tasks()
subjects = layout.get_subjects()

In [10]:
#Build a CSV to indicate columns in participants.tsv
task_run_dict = {}
for t in tasks:
    #For each task, get all scans unique without subject name
    task_files = layout.get(task=t,extension='nii.gz',space='T1w', suffix='bold')    
    
    break

In [34]:
f = layout.get(extension='nii.gz', space='T1w', suffix='bold')
a = [x for x in layout.get(extension='nii.gz',suffix='T1w') if x.filename.split('_')[2] == 'T1w.nii.gz']
f = list(set(['_'.join(x.filename.strip('.nii.gz').split('_')[1:]) for x in f]))
a = set(['_'.join(x.filename.strip('.nii.gz').split('_')[1:]) for x in a])

In [35]:
i = ignore_fields[0]
e = f[0]

In [38]:
import re

In [59]:
pattern = re.compile(r"{}-.*?(?=_)".format(i))

In [62]:
new_list = []
for k in f:
    
    try:
        match = pattern.findall(k)[0]
    except IndexError:
        new_list.append(k)
    
    new_list.append( k.replace('_{}'.format(match),''))

In [64]:
set(new_list)

{'ses-01_task-emp_run-01_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_run-02_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_run-03_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_run-1_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_run-2_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_run-3_space-T1w_desc-preproc_bold',
 'ses-01_task-emp_space-T1w_desc-preproc_bold',
 'ses-01_task-imi_run-01_space-T1w_desc-preproc_bold',
 'ses-01_task-imi_space-T1w_desc-preproc_bold',
 'ses-01_task-obs_run-01_space-T1w_desc-preproc_bold',
 'ses-01_task-obs_space-T1w_desc-preproc_bold',
 'ses-01_task-rest_run-01_space-T1w_desc-preproc_bold',
 'ses-01_task-rest_space-T1w_desc-preproc_bold',
 'ses-02_task-emp_run-01_space-T1w_desc-preproc_bold',
 'ses-02_task-emp_run-02_space-T1w_desc-preproc_bold',
 'ses-02_task-emp_run-1_space-T1w_desc-preproc_bold',
 'ses-02_task-emp_run-2_space-T1w_desc-preproc_bold',
 'ses-02_task-emp_run-3_space-T1w_desc-preproc_bold',
 'ses-02_task-imi_run-01_space-T1w_desc-pr

In [ ]:
#Loop through subjects, and get their seg_brainmask + t1_2_mni
brainmask_html = []
t12mni_html = []
missing_figs = []
missing_brainmask = []
missing_t12mni = []
for s in subjects:
    
    #Get figure directory and svgs
    figdir = os.path.join(fmriprep_dir,'sub-' + s,'figures')
    try:
        svgs = os.listdir(figdir)
    except FileNotFoundError:
        missing_figs.append(s)
        continue
    
    #Get brainmask and t12mni
    try:
        brainmask = [f for f in svgs if '_seg_brainmask' in f][0]
    except IndexError:
        missing_brainmask.append(s)
        continue
    brainmask = os.path.join(figdir,brainmask)

    try:
        t12mni = [f for f in svgs if 't1_2_mni' in f][0]
    except IndexError:
        missing_t12mni.append(s)
        continue
        
    t12mni = os.path.join(figdir,t12mni)
    
    brainmask_html += ['<tr><td>' + s + '</td></tr>']
    brainmask_html += ['<tr><td><object type="image/svg+xml" data="{}"></object>'.format(brainmask) + '</td></tr>' ]
    t12mni_html += ['<tr><td>' + s + '</td></tr>']
    t12mni_html += ['<tr><td><object type="image/svg+xml" data="{}"></object>'.format(brainmask) + '</td></tr>' ]

In [ ]:
with open('../output/brainmask.html','w') as f:
    f.writelines(brainmask_html)
    
with open('../output/t12mni.html','w') as f:
    f.writelines(t12mni_html)

In [ ]:
def get_svg(sub,ses,task,run,figtype,flist):
    
    for f in flist:
        #Remove extension
        splitted = f.split('.')[0].split('_')
        
        #All checks
        sub_in = ('sub-'+str(sub)) in splitted
        ses_in = ('ses-'+str(ses)) in splitted
        task_in = ('task-'+str(task)) in splitted
        
        if run:
            run_in = (('run-'+str(run)) in splitted) or (('run-0'+str(run)) in splitted)
        else:
            run_in = True
            
        figtype_in = figtype in splitted
        
        if sub_in and ses_in and task_in and run_in and figtype_in:
            return f

In [ ]:
os.path.basename(pages_dir)

In [ ]:
def make_html(tuplist,output_dir,group):
    html = []
    counter = 0
    page_ind = 0
    pages = []
    
    #Make pages directory for modality
    pages_dir = os.path.join(output_dir,group,'pages')
    try:
        os.makedirs(pages_dir,exist_ok=True)
    except OSError:
        pass

    #Chunk into 20
    for s,f in tuplist:
        
        if counter == 20:
            
            #Add next page item to previous?
            
            page = os.path.join(pages_dir,'{}.html'.format(page_ind))
            with open(page,'w') as p:
                p.writelines(html)
            html = []
            pages.append('{}.html'.format(page_ind))
            page_ind += 1
            counter = 0 
                
        
        html += ['<tr><td>' + s + '</td></tr>']
        html += ['<tr><td><object type="image/svg+xml" data="{}"></object>'.format(f) + '</td></tr>' ]
        counter+=1
        
    
    #Make an html page index
    index_html = []
    for i,p in enumerate(pages):
        index_html += ['<tr><td> <a href="./pages/{}">Page {}</a> </td></tr>'.format(p,i)]
        
    index_path = os.path.join(output_dir,group,'index.html')
    with open(index_path,'w') as f:
        f.writelines(index_html)
        

In [ ]:
#Now loop through tasks
for t in tasks:
    epi2t1 = []
    sdc = []
    rois = []
    
    missing_e2t = []
    missing_sdc = []
    missing_roi = []
    #For each task, get a list of each task SVG... 
    files = layout.get(task=t, suffix='bold',
                      space='T1w', extension='nii.gz')
    
    #For each file, put together pointer to SVG directory and get task files
    for f in files:
        sub = f.entities['subject']
        ses = f.entities['session']
        try:
            run = f.entities['run']
        except:
            run = False
        
        figdir = os.path.join(fmriprep_dir,'sub-' + sub,'figures')
        svgs = os.listdir(figdir)
        
        epi2t1_svg = get_svg(sub,ses,t,run,'bbregister',svgs) or get_svg(sub,ses,t,run,'coreg',svgs)
        if epi2t1_svg:
            epi2t1.append( (f.filename,os.path.join(figdir,epi2t1_svg)) )
        else:
            missing_e2t.append(f)
        
        sdc_svg = get_svg(sub,ses,t,run,'sdc',svgs)
        if sdc_svg:
            sdc.append((f.filename,os.path.join(figdir,sdc_svg)))
        else:
            missing_sdc.append(f)
        
        rois_svg = get_svg(sub,ses,t,run,'rois',svgs)
        if rois_svg:
            rois.append((f.filename,os.path.join(figdir,rois_svg)))  
        else:
            missing_roi.append(f)
        
    make_html(epi2t1,output,'{}_epi2t1'.format(t))
    make_html(sdc,output,'{}_sdc'.format(t))
    make_html(rois,output,'{}_rois'.format(t))

In [ ]:
#Try building an html template just using this
brainmask_html = []
t1_2_epi_html = [] 
sdc_html = []

for s in sub_dir:
    
    #Get figures directory
    subfig = os.path.join(sub_dir,'figures')
    
    #Get brainmask fig
    mask_svg = [f for f in os.listdir(subfig) if 'brainmask' in f]
    t12epi_svg = [f for f in os.listdir(subfig) if 'bbregister' in f]]
    sdc_svg = [f for f in os.listdir()]
    
    #Start new row
    html += ['<tr>']
    
    #Add subject ID
    html += ['<td>' + f + '</td>']
    
    #Add image
    img_path = os.path.relpath(os.path.join(subfig,f))
    html += ['<td> <object type="image/svg+xml" data="{}"></object>'.format(img_path) + '</td>']
    
    html += ['</tr>']

In [ ]:
with open('../output/test.html','w') as f:
    f.writelines(html)